In [5]:
import tensorrt as trt

In [6]:
import onnxruntime as ort

onnx_model_path = "./yolon_no_prune.onnx"
# session = ort.InferenceSession(onnx_model_path, providers=['CUDAExecutionProvider']))
# session = ort.InferenceSession(onnx_model_path, providers=['CPUExecutionProvider']))
session = ort.InferenceSession(onnx_model_path, providers=['TensorrtExecutionProvider'])

/home/RRAM_HKU/anaconda3/envs/DAC/lib/python3.9/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'TensorrtExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [7]:
image_folder_path = "./JPEGImages"

In [8]:
import numpy as np
import torch
from torchvision.ops import nms
from PIL import ImageDraw, ImageFont, Image

def dist2bbox(distance, anchor_points, xywh=True, dim=-1):
    """Transform distance(ltrb) to box(xywh or xyxy)."""
    # 左上右下
    lt, rb  = torch.split(distance, 2, dim)
    x1y1    = anchor_points - lt
    x2y2    = anchor_points + rb
    if xywh:
        c_xy    = (x1y1 + x2y2) / 2
        wh      = x2y2 - x1y1
        return torch.cat((c_xy, wh), dim)  # xywh bbox
    return torch.cat((x1y1, x2y2), dim)  # xyxy bbox


def decode_box(num_classes, input_shape, dbox, cls, anchors, strides):
    # dbox, cls, origin_cls, anchors, strides = inputs
    dbox = dist2bbox(dbox, anchors.unsqueeze(0), xywh=True, dim=1) * strides
    y = torch.cat((dbox, cls.sigmoid()), 1).permute(0, 2, 1)
    y[:, :, :4] = y[:, :, :4] / torch.Tensor([input_shape[1], input_shape[0], input_shape[1], input_shape[0]]).to(y.device)
    return y

def yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape, letterbox_image):
    box_yx = box_xy[..., ::-1]
    box_hw = box_wh[..., ::-1]
    input_shape = np.array(input_shape)
    image_shape = np.array(image_shape)

    if letterbox_image:
        new_shape = np.round(image_shape * np.min(input_shape/image_shape))
        offset = (input_shape - new_shape)/2./input_shape
        scale = input_shape/new_shape

        box_yx = (box_yx - offset) * scale
        box_hw *= scale

    box_mins = box_yx - (box_hw / 2.)
    box_maxes = box_yx + (box_hw / 2.)
    boxes = np.concatenate([box_mins[..., 0:1], box_mins[..., 1:2], box_maxes[..., 0:1], box_maxes[..., 1:2]], axis=-1)
    boxes *= np.concatenate([image_shape, image_shape], axis=-1)
    return boxes

def non_max_suppression(prediction, num_classes, input_shape, image_shape, letterbox_image, conf_thres=0.5, nms_thres=0.4):
    box_corner = prediction.new(prediction.shape)
    box_corner[:, :, 0] = prediction[:, :, 0] - prediction[:, :, 2] / 2
    box_corner[:, :, 1] = prediction[:, :, 1] - prediction[:, :, 3] / 2
    box_corner[:, :, 2] = prediction[:, :, 0] + prediction[:, :, 2] / 2
    box_corner[:, :, 3] = prediction[:, :, 1] + prediction[:, :, 3] / 2
    prediction[:, :, :4] = box_corner[:, :, :4]

    output = [None for _ in range(len(prediction))]
    for i, image_pred in enumerate(prediction):
        class_conf, class_pred = torch.max(image_pred[:, 4:4 + num_classes], 1, keepdim=True)
        conf_mask = (class_conf[:, 0] >= conf_thres).squeeze()
        image_pred = image_pred[conf_mask]
        class_conf = class_conf[conf_mask]
        class_pred = class_pred[conf_mask]
        if not image_pred.size(0):
            continue
        detections = torch.cat((image_pred[:, :4], class_conf.float(), class_pred.float()), 1)
        unique_labels = detections[:, -1].cpu().unique()

        if prediction.is_cuda:
            unique_labels = unique_labels.cuda()
            detections = detections.cuda()

        for c in unique_labels:
            detections_class = detections[detections[:, -1] == c]
            keep = nms(detections_class[:, :4], detections_class[:, 4], nms_thres)
            max_detections = detections_class[keep]
            output[i] = max_detections if output[i] is None else torch.cat((output[i], max_detections))
        
        if output[i] is not None:
            output[i] = output[i].cpu().numpy()
            box_xy, box_wh = (output[i][:, 0:2] + output[i][:, 2:4])/2, output[i][:, 2:4] - output[i][:, 0:2]
            output[i][:, :4] = yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape, letterbox_image)
    return output

    

In [9]:
input_shape = [640, 640]
num_classes = 7

In [10]:
import os
import time
import cv2
import numpy as np
from tqdm import tqdm  # Import tqdm for the progress bar

# Assuming you have already loaded the model and created a session
# session = ort.InferenceSession(onnx_model_path)

# Function to preprocess the image (modify as per your model's requirement)
def preprocess_input(image):
    image /= 255.0
    return image

def resize_image(image, size, letterbox_image):
    iw, ih  = image.size
    w, h    = size
    if letterbox_image:
        scale   = min(w/iw, h/ih)
        nw      = int(iw*scale)
        nh      = int(ih*scale)

        image   = image.resize((nw,nh), Image.BICUBIC)
        new_image = Image.new('RGB', size, (128,128,128))
        new_image.paste(image, ((w-nw)//2, (h-nh)//2))
    else:
        new_image = image.resize((w, h), Image.BICUBIC)
    return new_image

# Function to preprocess the image (modify as per your model's requirement)
def preprocess_image(image_path):
    img = Image.open(image_path)
    img  = resize_image(img, (input_shape[1], input_shape[0]), True)
    image_data  = np.expand_dims(np.transpose(preprocess_input(np.array(img, dtype='float32')), (2, 0, 1)), 0)
    return image_data

# List all images in the folder
image_paths = [os.path.join(image_folder_path, f) for f in os.listdir(image_folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

image_paths = image_paths[:1000]

total_time = 0

# Process each image and perform inference with a progress bar
for image_path in tqdm(image_paths, desc="Processing images"):
    input_image = preprocess_image(image_path)
    # Assuming the model takes an input named 'input' and outputs a tensor named 'output'
    image_shape = np.array(np.shape(input_image)[0:2])
    start_time = time.time()
    outputs = session.run(None, {'input': input_image})
    outputs = [torch.tensor(arr) for arr in outputs]
    #0是对的，1是对的，6对4，5对3
    outputs = decode_box(num_classes, input_shape, outputs[0], outputs[1], outputs[5], outputs[6])
    results = non_max_suppression(outputs, num_classes, input_shape, 
                image_shape, True, conf_thres = 0.5, nms_thres = 0.3)
    if results[0] is None: continue
    else:
        top_label   = np.array(results[0][:, 5], dtype = 'int32')
        top_conf    = results[0][:, 4]
        top_boxes   = results[0][:, :4]
        print(image_path)
        print(f'top_label: {top_label}')
        print(f'top_conf: {top_conf}')
        print(f'top_boxes: {top_boxes}')

    end_time = time.time()
    total_time = total_time + (end_time - start_time)


fps = len(image_paths) / total_time

print(f"Processed {len(image_paths)} images in {total_time:.2f} seconds.")
print(f"Frame per second (FPS): {fps:.2f}")

Processing images:   0%|          | 1/1000 [00:00<02:45,  6.05it/s]

./JPEGImages/00573.jpg
top_label: [0 0 0 0 0]
top_conf: [0.8689759  0.86187    0.8354737  0.78935987 0.72794044]
top_boxes: [[0.4849092  0.03379521 0.7847734  0.44397077]
 [0.45339012 0.38389036 0.67102414 0.64482147]
 [0.4214808  1.3755767  0.59858304 1.589336  ]
 [0.46671888 0.60648525 0.6318096  0.8016826 ]
 [0.48601568 0.7769531  0.5890289  0.88378096]]


Processing images:   0%|          | 3/1000 [00:00<02:54,  5.73it/s]

./JPEGImages/00537.jpg
top_label: [0]
top_conf: [0.67843604]
top_boxes: [[0.58772284 1.4504564  0.6382312  1.5062678 ]]


Processing images:   0%|          | 4/1000 [00:00<03:07,  5.32it/s]

...

./JPEGImages/00906.jpg
top_label: [1 1]
top_conf: [0.5732508  0.51475537]
top_boxes: [[0.8082503  1.3394376  0.9182454  1.3917282 ]
 [0.7938404  1.1996479  0.86024374 1.2489051 ]]
./JPEGImages/00899.jpg
top_label: [0 0 0]
top_conf: [0.7917786  0.6738785  0.62687254]
top_boxes: [[0.69330573 0.2682151  0.8026568  0.4584801 ]
 [0.7322937  0.45291162 0.79213864 0.6093724 ]
 [0.7789151  1.0811682  0.8330254  1.1617699 ]]


Processing images:  70%|███████   | 702/1000 [01:37<00:58,  5.11it/s]

./JPEGImages/00320.jpg
top_label: [1]
top_conf: [0.8144912]
top_boxes: [[0.49820852 2.4468265  0.597593   2.5117898 ]]


Processing images:  70%|███████   | 702/1000 [01:37<00:41,  7.22it/s]


KeyboardInterrupt: 